In [1]:
import glob
import random
import numpy as np
from scipy import signal
import json
import os
import sklearn
from sklearn import model_selection

In [2]:
import keras
from keras.models import Model,Sequential
from keras.layers import Dense, Activation, Permute, Dropout
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,UpSampling2D
from keras.layers import SeparableConv2D, DepthwiseConv2D
from keras.layers import BatchNormalization
from keras.layers import SpatialDropout2D
from keras.regularizers import l1_l2
from keras.layers import Input, Flatten
from keras.constraints import max_norm
from keras import backend as K
from keras.models import model_from_json
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

Using TensorFlow backend.


In [3]:
#from tensorflow.keras.backend import tensorflow_backend
import tensorflow
#dir(tensorflow.keras.backend)

In [4]:
batch_perc = 0.1 # Use 10 at each batch
num_epochs = 1
hp = np.load('hyper_parameters.npy',allow_pickle=True)[()]
h = 1 #int(sys.argv[1])
avg_method = 1 #int(sys.argv[2])
experiment = 'ECC_model'+str(h)+'_avg'+str(avg_method)

In [5]:
os.mkdir('Checkpoints/' + experiment)                       # make the directory '<time_val>'
os.mkdir('Checkpoints/' + experiment + '/topology')         # make the directory 'topology'
os.mkdir('Checkpoints/' + experiment + '/weights')          # make the directory for weights
os.mkdir('Checkpoints/' + experiment + '/performance')      # make the directory for performance

FileExistsError: [Errno 17] File exists: 'Checkpoints/ECC_model1_avg1'

In [6]:
def square(x):
    return K.square(x)
def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000)) 

In [7]:
def callbacks(experiment):
    ###########################################################################
    # Checkpoints 
    ###########################################################################
    filepath="Checkpoints/" + experiment + "/weights/nn_weights-{epoch:02d}.hdf5" # Where are checkpoints saved
    checkpoint = ModelCheckpoint(
                 filepath,
                 monitor='val_loss',                     # Validation set Loss           
                 verbose           = 0,                  # Display text 
                 save_weights_only = True,               # if True, only the model weights are saved
                 save_best_only    = False,              # if True, the latest-best model is overwritten
                 mode              = 'auto',             # used if 'save_best_only' is True  
                 save_freq         = 10)                 # Epochs between checkpoints
    return checkpoint

In [8]:
class MotorImageryDataset:
    def __init__(self, dataset='A01T.npz'):
        if not dataset.endswith('.npz'):
            dataset += '.npz'

        self.data = np.load(dataset)

        self.Fs = 250 # 250Hz from original paper

        # keys of data ['s', 'etyp', 'epos', 'edur', 'artifacts']

        self.raw = self.data['s'].T
        self.events_type = self.data['etyp'].T
        self.events_position = self.data['epos'].T
        self.events_duration = self.data['edur'].T
        self.artifacts = self.data['artifacts'].T

        # Types of motor imagery
        self.mi_types = {769: 'left', 770: 'right', 771: 'foot', 772: 'tongue', 783: 'unknown'}

    def get_trials_from_channel(self):

        # Channel default is C3

        startrial_code = 768
        starttrial_events = self.events_type == startrial_code
        idxs = [i for i, x in enumerate(starttrial_events[0]) if x]

        trials = []
        classes = []
        for index in idxs:
            #try:
            type_e = self.events_type[0, index+1]
            if type_e not in self.mi_types.keys():
                continue
            class_e = self.mi_types[type_e]
            if class_e == 'unknown':
                continue
            classes.append(type_e-769)

            start = self.events_position[0, index] + 0.5 * self.Fs
            stop = start + self.events_duration[0, index]
            if stop < start + 2* self.Fs:
                print(stop,start + 2* self.Fs)
                raise '(VVO error): EEG is shorter than 2 sec'
            #print(start,int(start + 2* self.Fs))
            trial = signal.resample(self.raw[0:22, int(start):int(start + 2* self.Fs)],2*128,axis=1)
            trials.append(trial.reshape(22,2*128,1))

            #except:
            #    continue

        return trials, classes

In [9]:
def group_trials(x_list,y_list,average=0,k=10):
    x_dict = {}
    for x,y in zip(x_list,y_list):
        if y not in x_dict.keys():
            x_dict[y] = []
        x_dict[y].append(x)
    out_list_x = []
    out_list_y = []
    out_list_l = []
    for y in x_dict.keys():
        #print(y,len(out_list_x))
        tmp_list_x = x_dict[y]
        tmp_list_y = []
        if average == 0:
            tmp_list_y = x_dict[y]
            random.shuffle(tmp_list_y)
        else:
            for ii,x in enumerate(tmp_list_x):
                tmp_list_y.append(np.mean(random.choices(tmp_list_x, k=k), axis=0))
        out_list_x.extend(tmp_list_x)
        out_list_y.extend(tmp_list_y)
        out_list_l.extend([y]*len(tmp_list_y))
    return out_list_x,out_list_y,out_list_l

In [10]:
trials = []
classes = []
for file in glob.glob('./bcidatasetIV2a/*.npz'):
    datasetA1 = MotorImageryDataset(file)
    tmp_trials, tmp_classes = datasetA1.get_trials_from_channel() # trials contains the N valid trials, and clases its related class.
    trials.extend(tmp_trials)
    classes.extend(tmp_classes)

In [11]:
c = list(zip(trials, classes))
random.shuffle(c)
trials, classes = zip(*c)

In [12]:
print('Trials shape:',np.shape(trials))

Trials shape: (2328, 22, 256, 1)


In [13]:
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(trials, classes, test_size=0.1, random_state=1)

In [14]:
print('(ECC) Number of training trials='+str(len(x_train))+'.\n(ECC) Number of validation trials='+str(len(x_val))+'.')

(ECC) Number of training trials=2095.
(ECC) Number of validation trials=233.


In [15]:
a,Chans,Samples,b = np.shape(trials)

In [16]:
input_main   = Input((1, Chans, Samples))

In [17]:
Samples = 64

In [18]:
nn = Sequential()
conv_sizes = []
# The encoder
for ii in range(hp['num_layers'][h]):
    if ii == 0:
        filter_shape = (22, 1)
    else:
        filter_shape = (11, 1)
    nn.add(Conv2D(hp['kernal_sizes'][h][ii], filter_shape,
                                     input_shape=(Chans, Samples,1),
                                     kernel_constraint = max_norm(2., axis=(0,1,2)),
                                     activation='tanh',
                                     padding='same',
                                     data_format       ='channels_last'))
    conv_sizes.append(nn.layers[-1].output_shape[3])
    if ii == 0:
        nn.add(MaxPooling2D((2, 4), padding='same'))
    else:
        nn.add(MaxPooling2D((1, 4), padding='same'))
        
    if hp['dropouts'][h][ii] > 0:
        nn.add(Dropout(hp['dropouts'][h][ii]))
        
    if hp['batch_norm'][h][ii] > 0 :
        nn.add(BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1))
        nn.add(BatchNormalization(axis=2, epsilon=1e-05, momentum=0.1))

In [19]:
# The decoder
for jj in range(hp['num_layers'][h],0,-1):
    ii = jj-1
    if ii == 0:
        filter_shape = (22, 1)
    else:
        filter_shape = (11, 1)
    nn.add(Conv2D(conv_sizes[ii], filter_shape,
                                     kernel_constraint = max_norm(2., axis=(0,1,2)),
                                     activation='tanh',
                                     padding='same',
                                     data_format       ='channels_last'))

    if ii == 0:
        nn.add(UpSampling2D((2, 1)))
    else:
        nn.add(UpSampling2D((1, 1)))
        
        
    if hp['dropouts'][h][ii] > 0:
        nn.add(Dropout(hp['dropouts'][h][ii]))
        
    if hp['batch_norm'][h][ii] > 0 :
        nn.add(BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1))
        nn.add(BatchNormalization(axis=2, epsilon=1e-05, momentum=0.1))

In [20]:
nn.add(Conv2D(1, (1, 1),
             kernel_constraint = max_norm(2., axis=(0,1,2)),
             activation='tanh',
             padding='same',
             data_format='channels_last'))

In [21]:
nn.build()
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 64, 64)        1472      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 11, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 16, 128)       90240     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 4, 128)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 11, 4, 128)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 4, 256)       

In [23]:
def create_repair_data(x_train,N=10,T1=32,T2=3):
    x_out = []
    y_out = []
    N = 10
    T1 = 2
    T2 = 3
    while len(x_out) <= N:
        x = random.choice(x_train)
        beginIdx = np.random.randint(x.shape[1]-(2*T1+T2))
        endIdx = beginIdx+2*T1+T2
        yIdx = beginIdx+T1+T2
        inputX  = np.hstack([x[:,beginIdx:beginIdx+T1,:],x[:,endIdx-T1:endIdx,:]]).reshape(x.shape[0],2,1)
        x_out.append(inputX)
        y_out.append(x[:,yIdx,:].reshape(x.shape[0],1,1))
    return x_out,y_out

In [24]:
x_t,y_t = create_repair_data(x_train)

In [27]:
x_t[0].shape

(22, 2, 1)

In [70]:
x_v,y_v = create_repair_data(x_val)

In [72]:
batch_size = int(round(batch_perc*len(x_train)))

In [73]:
grad_desc_algorithm = SGD(lr=hp['lr'][h], decay=0, momentum=hp['momentum'][h], nesterov=bool(hp['nesterov'][h]))

In [74]:
checkpoint = callbacks(experiment)

In [75]:
nn.compile(loss='mean_squared_error', optimizer=grad_desc_algorithm, metrics = ['accuracy'])

In [76]:
nn.fit(np.array(x_t),
         np.array(y_t),
         validation_data = (np.array(x_v),np.array(y_v)),
         epochs = num_epochs,
         initial_epoch = 0,
         batch_size = batch_size,
         verbose =1,
         callbacks=[checkpoint])

Train on 11 samples, validate on 11 samples
11/11 [==============================] - 3s 257ms/sample - loss: 143.7677 - accuracy: 0.0000e+00 - val_loss: 87.6242 - val_accuracy: 0.0000e+00
